In [1]:
import pandas as pd 
import requests
import wikipedia as wp
from bs4 import BeautifulSoup
import numpy as np
import io

In [4]:
%pip install geopy

    100% |████████████████████████████████| 102kB 1.3MB/s a 0:00:01
  Stored in directory: /home/nitinpa/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
Note: you may need to restart the kernel to use updated packages.


In [2]:
from geopy.geocoders import Nominatim
import folium

In [6]:
df_toronto = pd.read_csv('toronto_base.csv')
df_toronto.head()

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
latitude_toronto = 43.653963
longitude_toronto = 79.387207
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [48]:
CLIENT_ID  # your Foursquare ID
CLIENT_SECRET # your Foursquare Secret
VERSION

'20180605'

In [16]:
radius=500
LIMIT=100

,Unnamed: 0,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029


In [17]:
address_scar = 'Scarborough,Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774

In [18]:
map_scarb = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=12)

for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_scarb)  
    
map_scarb

In [26]:
neighborhood_latitude = scarborough_data.loc[0, 'Latitude']
neighborhood_longitude = scarborough_data.loc[0, 'Longitude']
neighborhood_name = scarborough_data.loc[0, 'Neighbourhood']

Latitude and longitude values of "Rouge, Malvern" are 43.8066863, -79.19435340000003.


In [27]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude_scar, longitude_scar, VERSION, radius, LIMIT)

In [28]:
results = requests.get(url).json()
# results

{'meta': {'code': 200, 'requestId': '5d4e47f66adbf5002c2fe70f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Scarborough City Centre',
  'headerFullLocation': 'Scarborough City Centre, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 43,
  'suggestedBounds': {'ne': {'lat': 43.7775770045, 'lng': -79.25155367954714},
   'sw': {'lat': 43.7685769955, 'lng': -79.26399432045285}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5085ec39e4b0b1ead2eb0818',
       'name': 'Disney Store',
       'location': {'address': '300 Borough Drive',
        'crossStreet': 'in Scarborough Town Centre',
        'lat': 43.775537,
        'lng': -79.256833,
        'labeledLa

In [29]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [30]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Disney Store,Toy / Game Store,43.775537,-79.256833
1,SEPHORA,Cosmetics Shop,43.775017,-79.258109
2,American Eagle Outfitters,Clothing Store,43.775908,-79.258352
3,Tommy Hilfiger Company Store,Clothing Store,43.776015,-79.257369
4,St. Andrews Fish & Chips,Fish & Chips Shop,43.771865,-79.252645


In [31]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighbourhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [33]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clairlea, Golden Mile, Oakridge",9,9,9,9,9,9
"Clarks Corners, Sullivan, Tam O'Shanter",10,10,10,10,10,10
"Cliffcrest, Cliffside, Scarborough Village West",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",6,6,6,6,6,6
"East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4


In [34]:
scarb_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")
scarb_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

fixed_columns = [scarb_onehot.columns[-1]] + list(scarb_onehot.columns[:-1])
scarb_onehot = scarb_onehot[fixed_columns]
scarb_grouped = scarb_onehot.groupby('Neighborhood').mean().reset_index()

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarb_grouped['Neighborhood']

for ind in np.arange(scarb_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarb_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Cosmetics Shop
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
3,Cedarbrae,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Convenience Store,Grocery Store
4,"Clairlea, Golden Mile, Oakridge",Bus Line,Soccer Field,Bakery,Intersection,Fast Food Restaurant,Bus Station,Park,Metro Station,Vietnamese Restaurant,Coffee Shop
5,"Clarks Corners, Sullivan, Tam O'Shanter",Pizza Place,Fried Chicken Joint,Thai Restaurant,Bank,Fast Food Restaurant,Italian Restaurant,Pharmacy,Noodle House,Chinese Restaurant,Vietnamese Restaurant
6,"Cliffcrest, Cliffside, Scarborough Village West",American Restaurant,Motel,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Cosmetics Shop
7,"Dorset Park, Scarborough Town Centre, Wexford ...",Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Sandwich Place,Soccer Field,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
8,"East Birchmount Park, Ionview, Kennedy Park",Chinese Restaurant,Department Store,Coffee Shop,Hobby Shop,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
9,"Guildwood, Morningside, West Hill",Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint


In [37]:
from sklearn.cluster import KMeans

scarb_data = scarborough_data.drop(16)
kclusters = 5
scarb_grouped_clustering = scarb_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarb_grouped_clustering)
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

In [38]:
scarb_merged = scarb_data
scarb_merged['Cluster Labels'] = kmeans.labels_
scarb_merged = scarb_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

In [39]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location = [latitude_scar, longitude_scar], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(scarb_merged['Latitude'], scarb_merged['Longitude'], scarb_merged['Neighbourhood'], scarb_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [40]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 0, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,-79.160497,0,Moving Target,Bar,Vietnamese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Cosmetics Shop
14,M1V,-79.284577,0,Playground,Park,Vietnamese Restaurant,Chinese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Cosmetics Shop


In [41]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 1, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,-79.194353,1,Fast Food Restaurant,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Department Store,Cosmetics Shop,Convenience Store
2,M1E,-79.188711,1,Intersection,Electronics Store,Rental Car Location,Breakfast Spot,Pizza Place,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Fried Chicken Joint
3,M1G,-79.216917,1,Coffee Shop,Korean Restaurant,Pharmacy,Vietnamese Restaurant,Clothing Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
4,M1H,-79.239476,1,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Lounge,Caribbean Restaurant,Hakka Restaurant,Convenience Store,Grocery Store
5,M1J,-79.239476,1,Playground,Convenience Store,Vietnamese Restaurant,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store
7,M1L,-79.284577,1,Bus Line,Soccer Field,Bakery,Intersection,Fast Food Restaurant,Bus Station,Park,Metro Station,Vietnamese Restaurant,Coffee Shop
8,M1M,-79.239476,1,American Restaurant,Motel,Hobby Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store,Cosmetics Shop
9,M1N,-79.264848,1,General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
11,M1R,-79.295849,1,Smoke Shop,Bakery,Breakfast Spot,Middle Eastern Restaurant,Vietnamese Restaurant,Coffee Shop,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
12,M1S,-79.262029,1,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Vietnamese Restaurant,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Department Store


In [42]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 2, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,M1T,-79.304302,2,Pizza Place,Fried Chicken Joint,Thai Restaurant,Bank,Fast Food Restaurant,Italian Restaurant,Pharmacy,Noodle House,Chinese Restaurant,Vietnamese Restaurant


In [43]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 3, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M1P,-79.273304,3,Indian Restaurant,Pet Store,Chinese Restaurant,Latin American Restaurant,Vietnamese Restaurant,Sandwich Place,Soccer Field,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [44]:
scarb_merged.loc[scarb_merged['Cluster Labels'] == 4, scarb_merged.columns[[1] + list(range(5, scarb_merged.shape[1]))]]

,Postcode,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,M1K,-79.262029,4,Chinese Restaurant,Department Store,Coffee Shop,Hobby Shop,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
